# Zaawansowane modele językowe 

In [3]:
import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch import nn

In [25]:
# device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
device = torch.device('mps')
print("Device:", device)

Device: mps


In [26]:
import transformers
from transformers import *
import numpy as np
from datasets import load_dataset
import evaluate
from tqdm import tqdm

# Bert

![title](https://i.pinimg.com/originals/d6/6a/3e/d66a3e867580854200fa37f08e8addaa.gif "segment")

In [27]:
model_name = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

loading configuration file config.json from cache at /Users/Kuba/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.25.1",
  "vocab_size": 30522
}

loading file vocab.txt from cache at /Users/Kuba/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/vocab.txt
loading file tokenizer.json from cache at /Users/Kuba/.cache/huggingface/hub/models--distilbert-b

In [28]:
tokenizer

PreTrainedTokenizerFast(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [29]:
model

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0): TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Linear(i

## Przetrenowane modele językowe


*   Huggingface: https://huggingface.co/transformers/pretrained_models.html
*   Community: https://huggingface.co/models



In [30]:
tokenized_sentence = tokenizer("BERT is designed to pre-train deep bidirectional representations")
tokenized_sentence

{'input_ids': [101, 14324, 2003, 2881, 2000, 3653, 1011, 3345, 2784, 7226, 7442, 7542, 2389, 15066, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [31]:
tokenizer.convert_ids_to_tokens(tokenized_sentence["input_ids"])

['[CLS]',
 'bert',
 'is',
 'designed',
 'to',
 'pre',
 '-',
 'train',
 'deep',
 'bid',
 '##ire',
 '##ction',
 '##al',
 'representations',
 '[SEP]']

In [32]:
tokenizer("Śpiulkolot")

{'input_ids': [101, 11867, 17922, 13687, 12898, 2102, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [33]:
tokenizer.convert_ids_to_tokens([101, 11867, 17922, 13687, 12898, 2102, 102])

['[CLS]', 'sp', '##iu', '##lk', '##olo', '##t', '[SEP]']

## Dotrenowywanie gotowego modelu

In [34]:
result = model(**tokenizer("BERT is designed to pre-train deep bidirectional representations by jointly conditioning on both left and right context in all layers.", return_tensors="pt"))
result.keys()

odict_keys(['last_hidden_state'])

In [35]:
result.last_hidden_state, result.last_hidden_state.size()

(tensor([[[-0.4867, -0.1770, -0.1706,  ..., -0.0752, -0.0195,  0.6925],
          [ 0.0744, -0.2203,  0.1330,  ..., -0.0730, -0.1128,  0.3024],
          [-0.6035, -0.2432, -0.0209,  ..., -0.2963, -0.6380,  0.6463],
          ...,
          [ 0.2347,  0.1218,  0.1038,  ..., -0.2239, -0.1634,  0.2523],
          [ 0.7587,  0.1439, -0.5647,  ...,  0.4042, -0.7747, -0.2532],
          [ 0.3978,  0.4038, -0.3577,  ...,  0.3608, -0.9130, -0.0176]]],
        grad_fn=<NativeLayerNormBackward0>),
 torch.Size([1, 28, 768]))

In [36]:
sentiment_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

loading configuration file config.json from cache at /Users/Kuba/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.25.1",
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /Users/Kuba/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/pytorch_model.bin
Some weights of the model checkpoint at distilbert-base-uncased were not 

In [37]:
sentiment_model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

### Training

https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments

In [38]:
raw_datasets = load_dataset("glue","sst2")

Found cached dataset glue (/Users/Kuba/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|██████████| 3/3 [00:00<00:00, 1273.32it/s]


In [39]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})

In [40]:
raw_datasets["train"]["sentence"]

['hide new secretions from the parental units ',
 'contains no wit , only labored gags ',
 'that loves its characters and communicates something rather beautiful about human nature ',
 'remains utterly satisfied to remain the same throughout ',
 'on the worst revenge-of-the-nerds clichés the filmmakers could dredge up ',
 "that 's far too tragic to merit such superficial treatment ",
 'demonstrates that the director of such hollywood blockbusters as patriot games can still turn out a small , personal film with an emotional wallop . ',
 'of saucy ',
 "a depressed fifteen-year-old 's suicidal poetry ",
 "are more deeply thought through than in most ` right-thinking ' films ",
 'goes to absurd lengths ',
 "for those moviegoers who complain that ` they do n't make movies like they used to anymore ",
 "the part where nothing 's happening , ",
 'saw how bad this movie was ',
 'lend some dignity to a dumb story ',
 'the greatest musicians ',
 'cold movie ',
 'with his usual intelligence and s

In [41]:
def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding="max_length", truncation=True)


In [42]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["sentence","idx"])
tokenized_datasets.set_format("torch")
small_train_dataset = tokenized_datasets["train"]#.shuffle(seed=42).select(range(10000))
small_eval_dataset = tokenized_datasets["validation"]

Loading cached processed dataset at /Users/Kuba/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-04686632094aa482.arrow
100%|██████████| 1/1 [00:00<00:00, 14.85ba/s]
Loading cached processed dataset at /Users/Kuba/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-cb12914be85827c8.arrow


In [43]:
train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=16)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=16)


In [44]:
optimizer = Adam(sentiment_model.parameters(), lr=5e-5)
sentiment_model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [ ]:
num_epochs = 3
loss_fun = nn.CrossEntropyLoss()

for epoch in range(num_epochs):
    losses = []
    for batch in tqdm(train_dataloader):

        labels = batch["label"].to(device)
        batch = {"attention_mask": batch['attention_mask'].to(device), "input_ids": batch['input_ids'].to(device)}#, "token_type_ids":batch['token_type_ids'].to(device)}
        outputs = sentiment_model(**batch)
        loss = loss_fun(outputs.logits, labels)
        loss.backward()

        optimizer.step()
#         lr_scheduler.step()
        optimizer.zero_grad()
#         progress_bar.update(1)
        losses.append(loss.item())
    print(np.mean(losses))

In [ ]:
sentiment_model.load_state_dict(torch.load("sentiment_model_dict"))

In [ ]:
metric = evaluate.load("accuracy")
model.eval()
for batch in eval_dataloader:
    labels = batch["label"].to(device)
    batch = {"attention_mask": batch['attention_mask'].to(device), "input_ids": batch['input_ids'].to(device)}#, "token_type_ids":batch['token_type_ids'].to(device)}

    with torch.no_grad():
        outputs = sentiment_model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=labels)

metric.compute()

# Odpowiadanie na pytania (Question answering)

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
model_name = "deepset/roberta-base-squad2"
qa_tokenizer = AutoTokenizer.from_pretrained("deepset/roberta-base-squad2")

qa_model = AutoModelForQuestionAnswering.from_pretrained("deepset/roberta-base-squad2")

In [ ]:
context = '''Warsaw is the capital and largest city of Poland. 
The metropolis stands on the River Vistula in east-central Poland and its population is officially estimated at 1.8 million 
residents within a greater metropolitan area of 3.1 million residents, which makes Warsaw the 7th 
most-populous capital city in the European Union. 
The city area measures 517 km2 (200 sq mi) and comprises 18 boroughs, 
while the metropolitan area covers 6,100 km2 (2,355 sq mi).
Warsaw is an alpha- global city, a major cultural, political and economic hub, 
and the country's seat of government. Its historical Old Town was designated a UNESCO World Heritage Site.'''

In [ ]:
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)
QA_input = {
    'question': 'Where does the water in Warsaw come from?',
    'context': context}
res = nlp(QA_input)

In [ ]:
res

# Generacja tekstu:
Pograj w grę: https://play.aidungeon.io/main/newGame (model GPT-3)

# Obrazki z tekstu
https://openai.com/dall-e-2/

https://labs.openai.com/

# Chatbot
https://openai.com/blog/chatgpt/

https://chat.openai.com/chat